In [1]:
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.nn.functional as F

import pytorch3d
import pytorch3d.io
from pytorch3d.structures import Meshes
from pytorch3d.renderer import TexturesVertex

import trimesh

from src.util import make_faces

In [2]:
n = 2
start = -0.5
end = 0.5

d1, d2 = torch.meshgrid(
    torch.linspace(start, end, steps=n), 
    torch.linspace(start, end, steps=n))
d3 = torch.full_like(d1, end) + 1 / n
sides =  OrderedDict({
    'front': torch.stack((+d3,  d1,  d2), dim=-1),
    'right': torch.stack(( d1, +d3,  d2), dim=-1),    
    'back' : torch.stack((-d3,  d1,  d2), dim=-1),         
    'left' : torch.stack(( d1, -d3,  d2), dim=-1),
    'top'  : torch.stack(( d1,  d2, +d3), dim=-1),
    'down' : torch.stack(( d1,  d2, -d3), dim=-1),
})
vert = torch.stack(list(sides.values())).reshape(-1, 3)
offset, faces = n ** 2, make_faces(n, n)
faces = torch.cat([
    i * offset + torch.tensor(faces)
    for i in range(6)])
textures = TexturesVertex(verts_features=[torch.ones_like(vert)])
mesh = Meshes(verts=[vert], faces=[faces], textures=textures)

corners = torch.tensor([
    [ 0, 22, 14],
    [ 3, 19,  7],
    
    [16,  9, 13],   
    [17, 11,  5],
    
    [ 1, 18, 15],
    [ 2, 23,  6],
    
    [20,  8, 12],
    [21, 10,  4],
])

m = trimesh.Trimesh(vertices=vert, 
    faces=torch.cat((faces, corners)))
m.export('./data/cube-edge.stl');

In [3]:
corners = torch.tensor([
    [ 0, 22, 14],
    [ 3, 19,  7],
    
    [16,  9, 13],   
    [17, 11,  5],
    
    [ 1, 18, 15],
    [ 2, 23,  6],
    
    [20,  8, 12],
    [21, 10,  4],
])
22 // 6, 22 % 6

(3, 4)

In [4]:
tris = torch.div(corners, n**2, rounding_mode='trunc')
tris

tensor([[0, 5, 3],
        [0, 4, 1],
        [4, 2, 3],
        [4, 2, 1],
        [0, 4, 3],
        [0, 5, 1],
        [5, 2, 3],
        [5, 2, 1]])

In [5]:
torch.remainder(corners, n**2)

tensor([[0, 2, 2],
        [3, 3, 3],
        [0, 1, 1],
        [1, 3, 1],
        [1, 2, 3],
        [2, 3, 2],
        [0, 0, 0],
        [1, 2, 0]])

In [6]:
n = 2
rmn = torch.tensor([[   0,   n,   n],
                    [ n**2-n, n**2-n, n**2-n],
                    [   0,   1,   1],
                    [   1, n**2-n,   1],
                    [   1,   n, n**2-n],
                    [   n, n**2-n,   n],
                    [   0,   0,   0],
                    [   1,   n,   0]])

print(rmn)

tensor([[0, 2, 2],
        [2, 2, 2],
        [0, 1, 1],
        [1, 2, 1],
        [1, 2, 2],
        [2, 2, 2],
        [0, 0, 0],
        [1, 2, 0]])


In [7]:
torch.allclose(rmn, torch.remainder(corners, n**2))

False

In [8]:
torch.allclose(corners, tris * n**2 + rmn)

False

In [10]:
n = 3
start = -0.5
end = 0.5

d1, d2 = torch.meshgrid(
    torch.linspace(start, end, steps=n), 
    torch.linspace(start, end, steps=n))
d3 = torch.full_like(d1, end) + 1 / n
sides =  OrderedDict({
    'front': torch.stack((+d3,  d1,  d2), dim=-1),
    'right': torch.stack(( d1, +d3,  d2), dim=-1),    
    'back' : torch.stack((-d3,  d1,  d2), dim=-1),         
    'left' : torch.stack(( d1, -d3,  d2), dim=-1),
    'top'  : torch.stack(( d1,  d2, +d3), dim=-1),
    'down' : torch.stack(( d1,  d2, -d3), dim=-1),
})
vert = torch.stack(list(sides.values())).reshape(-1, 3)
offset, faces = n ** 2, make_faces(n, n)
faces = torch.cat([
    i * offset + torch.tensor(faces)
    for i in range(6)])
textures = TexturesVertex(verts_features=[torch.ones_like(vert)])
mesh = Meshes(verts=[vert], faces=[faces], textures=textures)

n = 3
c0 = 0
c1 = n-1
c2 = n**2-n
c3 = n**2-1 
rmn = torch.tensor(
[[c0, c2, c2],
 [c3, c3, c3],
 [c0, c1, c1],
 [c1, c3, c1],
 [c1, c2, c3],
 [c2, c3, c2],
 [c0, c0, c0],
 [c1, c2, c0]])
rmn

corners = tris * n**2 + rmn

m = trimesh.Trimesh(vertices=vert, 
    faces=torch.cat((faces, corners)))
m.export('./data/cube-3x3.stl');

In [11]:
corners

tensor([[ 0, 51, 33],
        [ 8, 44, 17],
        [36, 20, 29],
        [38, 26, 11],
        [ 2, 42, 35],
        [ 6, 53, 15],
        [45, 18, 27],
        [47, 24,  9]])

In [12]:
# 0 1
# 2 3
# 0 1 2
# 3 4 5
# 6 7 8
3**2 -1

8

In [13]:
torch.remainder(corners, n**2)

tensor([[0, 6, 6],
        [8, 8, 8],
        [0, 2, 2],
        [2, 8, 2],
        [2, 6, 8],
        [6, 8, 6],
        [0, 0, 0],
        [2, 6, 0]])

In [14]:
n = 3
c0 = 0
c1 = n-1
c2 = n**2-n
c3 = n**2-1 
rs = torch.tensor(
[[c0, c2, c2],
 [c3, c3, c3],
 [c0, c1, c1],
 [c1, c3, c1],
 [c1, c2, c3],
 [c2, c3, c2],
 [c0, c0, c0],
 [c1, c2, c0]])
rs

tensor([[0, 6, 6],
        [8, 8, 8],
        [0, 2, 2],
        [2, 8, 2],
        [2, 6, 8],
        [6, 8, 6],
        [0, 0, 0],
        [2, 6, 0]])

In [ ]:
rs - torch.remainder(corners, n**2)